In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'https://www.iisue.unam.mx/perfiles/numeros-anteriores'
url_root = 'https://www.iisue.unam.mx/perfiles/numeros-anteriores'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='container')
volumen=box.findAll('div', class_='col-4 col-md-3 col-lg-2')
vol = [x.find('a').get('href')for x in volumen]
vol = [urljoin (url_root, i) for i in vol]
vol

['https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4494',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4488',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4489',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4490',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4492',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4493',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4483',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4484',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4485',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4487',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4478',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/issue/view/4479',
 'ht

In [55]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    #box1 = soup1.find('div', class_='sections') 
    volumen1=soup1.findAll('div', class_='obj_article_summary')
    vol1= [o.find('a').get('href')for o in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1

In [56]:
vol2

['https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/61024',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60407',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60009',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60397',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60049',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/59994',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60166',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/59898',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60179',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60012',
 'https://perfileseducativos.unam.mx/iisue_pe/index.php/perfiles/article/view/60178',
 'https://perfileseducativos.unam.mx/iisue_pe/index.ph

In [57]:
def get_url_items(sopa,url):
    volumen1=soup1.findAll('div', class_='obj_article_summary')
    vol1= [o.find('a').get('href')for o in volumen1]
    vol1= [o.get('href')for o in volumen1]
    return vol2

In [58]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.iisue.unam.mx/perfiles/numeros-anteriores


In [59]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

729

In [60]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [61]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perfiles Educativos'
    a='V. Ciencias Sociales'
    tem='Investigación Educativa'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='item abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='icon-menu').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [62]:
list_scraper=list_scraper[0:729]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [63]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,2022: nuevas y emergentes superficies de inscr...,None,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
1,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,Aproximaciones a la temprana formación cívica ...,Entre los eventos más significativos que la mo...,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
2,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,Las desigualdades socioeducativas en el espaci...,El artículo tiene el objetivo de describir la ...,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
3,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,Universidades públicas mexicanas,Se analiza el Programa de Carrera Docente en U...,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
4,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,La indagación colaborativa,En este artículo se describe la implementación...,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
...,...,...,...,...,...,...,...,...
724,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,(Re)crear la universidad: una premisa urgente,"Este estudio, de cuño bibliográfico, se propon...",None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
725,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,CMES 2009 Las nuevas dinámicas de la educación...,None,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
726,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,Currículum: experiencias y configuraciones con...,None,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...
727,Perfiles Educativos,V. Ciencias Sociales,Investigación Educativa,Las maestras en México. Re-cuento de una historia,None,None,https://perfileseducativos.unam.mx/iisue_pe/in...,https://perfileseducativos.unam.mx/iisue_pe/in...


In [64]:
df_catalogo.to_csv('Revista051.csv', index=False)